In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv('/kaggle/input/musicnet-dataset/musicnet/musicnet/train_labels/1727.csv')
data

,start_time,end_time,instrument,note,start_beat,end_beat,note_value
0,9182,90078,43,53,4.000,1.500,Dotted Quarter
1,9182,33758,42,65,4.000,0.500,Eighth
2,9182,62430,1,69,4.000,1.000,Quarter
3,9182,202206,44,41,4.000,3.500,Whole
4,9182,62430,1,81,4.000,1.000,Quarter
...,...,...,...,...,...,...,...
6575,19196894,19421150,44,29,365.000,3.000,Dotted Half
6576,19226590,19233758,1,60,365.375,0.125,Thirty Second
6577,19226590,19233758,1,48,365.375,0.125,Thirty Second
6578,19233758,19421150,1,65,365.500,2.500,Dotted Half


In [3]:
pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 73.6 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=140702cd0346509db5df6096f94843d1d154a4c978e1ddab9a730c912add2472
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 require

In [4]:
import pretty_midi

In [5]:
import os
import pretty_midi

entire_data = []

def midi_data_extraction(file_path):
    # Check if the path is a directory
    if os.path.isdir(file_path):
        # Loop through all files in the directory
        for file_name in os.listdir(file_path):
            if file_name.endswith('.mid'):
                # Construct the full file path
                full_path = os.path.join(file_path, file_name)
                # Load the MIDI file
                midi_data = pretty_midi.PrettyMIDI(full_path)
                
                # Extract global features
                tempo = midi_data.estimate_tempo()  # Average tempo
                time_signature = midi_data.time_signature_changes  # Time signature changes

                # Extract data for each note
                for instrument in midi_data.instruments:
                    for note in instrument.notes:
                        start_time = note.start
                        end_time = note.end
                        instrument_name = instrument.name if instrument.name else 'Unnamed'
                        note_name = pretty_midi.note_number_to_name(note.pitch)
                        note_value = note.pitch
                        duration = end_time - start_time  # Duration of the note
                        velocity = note.velocity  # Velocity of the note
                        
                        # Calculate time in beats
                        start_beat = midi_data.get_beats(start_time)
                        end_beat = midi_data.get_beats(end_time)
                        
                        # Append data to the list
                        entire_data.append({
                            'file_name': file_name,
                            'instrument_name': instrument_name,
                            'note_name': note_name,
                            'note_value': note_value,
                            'start_time': start_time,
                            'end_time': end_time,
                            'duration': duration,
                            'velocity': velocity,
                            'start_beat': start_beat,
                            'end_beat': end_beat,
                            'tempo': tempo,
                            'time_signature': str(time_signature[0]) if time_signature else '4/4'  # Default to 4/4 if not available
                        })

    return entire_data


In [6]:
import tensorflow as tf

# Force TensorFlow to use GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print("GPU is being used.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available, using CPU.")


2024-08-11 03:37:39.319655: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 03:37:39.319773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 03:37:39.455882: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU is being used.


In [ ]:
file_name = '/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Beethoven'
midi_data_extraction(file_name)

/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [ ]:
columns = ['file_name', 'instrument_name', 'note_name', 'note_value', 'start_time', 'end_time', 'duration', 'velocity', 'start_beat', 'end_beat', 'tempo', 'time_signature']

# Create a DataFrame with the collected data
df = pd.DataFrame(entire_data, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv('entire_midi_data.csv', index=False)

# Print the first few rows of the DataFrame
print(df.head())


In [ ]:
with open('midi_text_data.txt', 'w') as file:
    file.write('\n')
    for index, row in df.iterrows():
        text_line = (
            f"file_name: {row['file_name']}, "
            f"instrument_name: {row['instrument_name']}, "
            f"note_name: {row['note_name']}, "
            f"note_value: {row['note_value']}, "
            f"start_time: {row['start_time']}, "
            f"end_time: {row['end_time']}, "
            f"duration: {row['duration']}, "
            f"velocity: {row['velocity']}, "
            f"start_beat: {row['start_beat']}, "
            f"end_beat: {row['end_beat']}, "
            f"tempo: {row['tempo']}, "
            f"time_signature: {row['time_signature']}\n"
        )
        file.write(text_line)
    file.write('\n')


# Model 

In [ ]:
pip install transformers

In [ ]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import TextDataset, Trainer, TrainingArguments

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )
    return dataset

In [ ]:
train_dataset = load_dataset('midi_text_data.txt', tokenizer)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

In [ ]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

# Testing 

In [ ]:
import pretty_midi

def midi_to_text(file_path):
    midi_data = pretty_midi.PrettyMIDI(file_path)
    lines = []
    
    for instrument in midi_data.instruments:
        instrument_name = instrument.name if instrument.name else 'Unnamed'
        for note in instrument.notes:
            start_time = note.start
            end_time = note.end
            note_name = pretty_midi.note_number_to_name(note.pitch)
            start_beat = midi_data.time_to_tick(start_time)
            end_beat = midi_data.time_to_tick(end_time)
            note_value = note.pitch
            duration = end_time - start_time
            velocity = note.velocity

            line = (
                f"instrument_name: {instrument_name}, "
                f"note_name: {note_name}, "
                f"note_value: {note_value}, "
                f"start_time: {start_time}, "
                f"end_time: {end_time}, "
                f"duration: {duration}, "
                f"velocity: {velocity}, "
                f"start_beat: {start_beat}, "
                f"end_beat: {end_beat}\n"
            )
            lines.append(line)
    
    return "\n".join(lines)



In [ ]:
# Path to your MIDI file
midi_file_path = '/kaggle/input/chopin/your_midi_file.mid'
text_data = midi_to_text(midi_file_path)

# Save the extracted text data
with open('chopin_text_data.txt', 'w') as file:
    file.write(text_data)


In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model = TFGPT2LMHeadModel.from_pretrained('./fine_tuned_gpt2')  # Make sure this path is correct
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_gpt2')


In [ ]:
# Use the extracted text as input
input_text = text_data  # The text data extracted from the MIDI file

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors='tf')

# Generate music (text sequence) using the fine-tuned model
output = model.generate(input_ids, max_length=500, num_return_sequences=1)

# Decode the generated sequence to text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)


In [ ]:
def text_to_midi(text):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # Default to piano

    for line in text.split('\n'):
        if line:
            parts = line.split(',')
            note_value = int(parts[2].split(': ')[1])
            start_time = float(parts[3].split(': ')[1])
            end_time = float(parts[4].split(': ')[1])
            velocity = int(parts[6].split(': ')[1])

            note = pretty_midi.Note(velocity=velocity, pitch=note_value, start=start_time, end=end_time)
            instrument.notes.append(note)
    
    midi.instruments.append(instrument)
    return midi

# Convert the generated text back to MIDI
midi_output = text_to_midi(generated_text)

# Save the new MIDI file
midi_output.write('generated_music.mid')
